### Loading libs and data

In [183]:
import currencyapicom
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import warnings
import mysql.connector 
from datetime import timedelta
import time


In [128]:
warnings.filterwarnings('ignore')

In [270]:
client = currencyapicom.Client('cur_live_MZNLOlCwTpr2N7OB6DFle53JTQnYaDxVbHoY0Qmc')

In [2]:
df_data = pd.read_csv('/Users/leox_50/Desktop/KSE/Job preparation/OBRIO/test_payment_data.csv')

In [3]:
ProfileReport(df_data)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [316]:
result = client.historical('2021-12-31')

In [8]:
print(result.keys)

<built-in method keys of dict object at 0x178956b00>


In [341]:
connection = mysql.connector.connect(host='localhost',
                                         database='obrio_test',
                                         user='root',
                                         password='@Querto15')
if connection.is_connected():
    db_Info = connection.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    cursor = connection.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)


Connected to MySQL Server version  8.0.29
You're connected to database:  ('obrio_test',)


### First insert into the table

In [168]:
result = client.historical('2022-01-01')

In [166]:
def temp_data(max_date):
    
    result = client.historical(str(max_date.date() + timedelta(days=1)))
    
    return result

In [153]:
def preparing_data(result):
    
    data_list = [{'currency': key, **value} for key, value in result['data'].items()]

    currency_table = pd.DataFrame(data_list)
    currency_table['datetime'] = result['meta']['last_updated_at']
    currency_table['primary_key'] = currency_table['datetime'].astype(str) + '-' + currency_table['code']

    return currency_table

In [170]:
def primary_key_checker(data):

    query = "SELECT primary_key FROM obrio_test.currencies_exchange_rate"

    all_data = pd.read_sql(query, connection)
    data_upd = data[data['primary_key'].isin(all_data['primary_key']) == False]
    dups = data.shape[0] - data_upd.shape[0] 

    if dups > 0:
        return print(f"There are {dups} entrances already existing. Check {data['datetime'].head(1)[0]} date")
    else:
        return data_upd

In [221]:
def insert_in_table(data):

    if isinstance(data, str):

        return data

    elif isinstance(data, pd.DataFrame):

        currency_table = data
    
        df_to_store = currency_table[['primary_key','code','value','datetime']].rename(columns = {'code':'currency_code','value':'value_to_usd'})
        df_to_store['primary_key'] = df_to_store['primary_key'].apply(lambda x: "'" + str(x) + "'")
        df_to_store['datetime'] = df_to_store['datetime'].apply(lambda x: "'" + str(x) + "'")
        df_to_store['currency_code'] = df_to_store['currency_code'].apply(lambda x: "'" + str(x) + "'")

    for _, row in df_to_store.iterrows():
        values = ", ".join([str(value) for value in row])
        cursor.execute(f"insert into obrio_test.currencies_exchange_rate (primary_key, currency_code, value_to_usd, datetime) values ({values})")
        connection.commit()

In [318]:
insert_in_table(
    primary_key_checker(
        preparing_data(result)
    )
)

### Updating the exsiting table

In [184]:
def updating_table(number_of_itterations:int):

    for i in range(0,number_of_itterations):
        
        query = "SELECT MAX(datetime) as datetime FROM obrio_test.currencies_exchange_rate"

        max_date = pd.read_sql(query, connection)
        max_date = pd.to_datetime(max_date['datetime']).iloc[0]

        result = client.historical(str(max_date.date() + timedelta(days=1)))

        data_list = [{'currency': key, **value} for key, value in result['data'].items()]

        currency_table = pd.DataFrame(data_list)
        currency_table['datetime'] = result['meta']['last_updated_at']
        currency_table['primary_key'] = currency_table['datetime'].astype(str) + '-' + currency_table['code']

        query = "SELECT primary_key FROM obrio_test.currencies_exchange_rate"

        all_data = pd.read_sql(query, connection)
        data_upd = currency_table[currency_table['primary_key'].isin(all_data['primary_key']) == False]
        dups = currency_table.shape[0] - data_upd.shape[0] 

        if dups > 0:
            return print(f"There are {dups} entrances already existing. Check {currency_table['datetime'].head(1)[0]} date")
        else:
        
            df_to_store = data_upd[['primary_key','code','value','datetime']].rename(columns = {'code':'currency_code','value':'value_to_usd'})
            df_to_store['primary_key'] = df_to_store['primary_key'].apply(lambda x: "'" + str(x) + "'")
            df_to_store['datetime'] = df_to_store['datetime'].apply(lambda x: "'" + str(x) + "'")
            df_to_store['currency_code'] = df_to_store['currency_code'].apply(lambda x: "'" + str(x) + "'")

            for _, row in df_to_store.iterrows():
                values = ", ".join([str(value) for value in row])
                cursor.execute(f"insert into obrio_test.currencies_exchange_rate (primary_key, currency_code, value_to_usd, datetime) values ({values})")
                connection.commit()
        
        time.sleep(6)

In [275]:
updating_table(50)

In [353]:
query = "SELECT * FROM obrio_test.currencies_exchange_rate"

all_dates = pd.read_sql(query, connection)

### Merging data from the database to the existing table

In [320]:
df_eur_usd = all_dates[all_dates['currency_code'].isin(('EUR','USD'))][['currency_code', 'value_to_usd','datetime']].rename(columns=  {'currency_code':'currency'})
df_eur_usd['event_time_join'] = pd.to_datetime(df_eur_usd['datetime']).dt.date

In [321]:
# as well as the currency in this service is updating only by the end of the day, the yesterday's values are more trustworthy in this case

df_data['event_time_join'] = (pd.to_datetime(df_data['event_time']) - timedelta(days=1)).dt.date

In [326]:
df_all_data = pd.merge(df_data,df_eur_usd, how = 'left', on = ['event_time_join','currency'])

In [330]:
df_eur = df_eur_usd[df_eur_usd['currency'] == 'EUR'][['event_time_join','value_to_usd']].rename(columns={'value_to_usd':'eur_usd'})

df_all_data = pd.merge(df_all_data, df_eur, how = 'left', on = 'event_time_join')

In [306]:
data = df_all_data[df_all_data['user_id'].isin(df_all_data[df_all_data['card_country'].isnull() == True]['user_id']) == True]
data['is_null'] = np.where(data['card_country'].isnull() == True, 'Yes','No')

In [287]:
def check_errors(param):

    return data[[param,'is_null','user_id']].groupby([param,'is_null']).count()

In [295]:
check_errors('error_type')

user_id
error_type is_null         
0.01       No             7
           Yes           77
2.01       No             1
           Yes            2
2.02       Yes            2
2.06       No             2
           Yes            4
2.08       No            19
           Yes           90
2.09       Yes            2
2.11       No             7
           Yes           12
3.01       No             2
           Yes            2
3.02       No            42
           Yes           37
3.04       Yes            9
3.07       No             2
           Yes           87
3.08       No             9
           Yes           37
3.10       No            18
           Yes           11
4.02       Yes            1
4.03       No            14
           Yes            5
4.05       No             7
           Yes            1
4.06       No             1
4.09       No            18
           Yes            4
5.01       No            19
           Yes            1
5.08       No            10
6.01       No             1
7.01       Yes            1

In [ ]:
# in case of a 2.08 error the data could not be recorded because of wrong curd number
# in case of 3.07 error the credentials could simply not be recorded

In [338]:
df_all_data.to_csv('updated_paym_data.csv')